# Training fast0tag to solve the image tagging problem
 
## What is the image tagging problem?
Suppose we have an image and we want to be able to describe that image. In the most basic sense we have image classification where we want to assign this image to one of a select number of pre-defined classes. This, however, is limited in its utility because in order to effectively distinguish between two classes we are generally not allowed to have any overlap between them.

This is where image tagging comes into play. We recognize that it is reasonable to have two completely different things share similar features. An example would be birds and airplanes sharing the similarity of flight, but it is not reasonable to group birds and airplanes together for most applications. So, we relax the reqirement of little-to-no overlap between classes leading us to class boundaries which are much more fuzzy.

This increased fuzziness of class boundaries leads to a theoretical question of how we can effectively assign these new types of labels (we will use the terms "tag" and "label" interchangeably).

As it turns out, we can take advantage of image similarities to images with known tags to find tags for new images. We can take this one step further and use linguistic similarities to assign tags that without providing any training examples for those tags! This problem is called **zero-shot learning** - the ability to assign labels without any prior training on those labels. Zero-shot learning is a special case of the more general problem, **transfer learning**, which aims to use "knowledge" gained by solving one problem and using it to solve a different problem.

## fast0tag
A relatively simple approach involves breaking down the image taggng problem into 3 sub-problems:
 1. Generate an "image-feature space" for containing relevant features for the images
 2. Generate a "word-feature space" that captures semantic and syntactic relations of words
 3. Find a mapping that assigns relevant tags/labels from the word space to images in the image space
 
The first and second have already been solved in numerous ways and we will borrow from existing literature in solving those. The third problem turns out to have a nice solution as was described by Zhang, et al in their paper outlining the algorithm they call **fast0tag**.

## Importing our data
For this implementation we will follow in the footsteps of the paper we are referrencing by using the NUS-WIDE dataset.

In [ ]:
import numpy as np
import pandas as pd

img_df = pd.load_csv("./train.csv")
tag_df = pd.load_csv("./tags.csv")

## Problem 1: Generating the image-feature space
This problem is well-documented and can be accomplished by training a Convolutional Neural Network. CNNs classify images by first learning complex features to describe the image before passing these features to a standard classifier to perform the desired classification. By removing the classification component, called the *fully-connected layer*, we can extract just the relevant image's features.

We want the upmost generality in our algorithm, so we can tag as many different types of images as possible. To accomplish this we will use a CNN architecture, VGG19, that was trained on images from the ImageNet competition.

The question arises on whether or not it would be a good idea to fine-tune VGG19 for the specific images we want to classify. We found that the generalization of the algorithm did not significantly improve, but if the application in mind only requires tagging certain types of images, it could be advantageous to fine-tune the CNN.

In [ ]:
from keras.applications.vgg19 import VGG19, preprocess_input
from keras.preprocessing import image

img_model = VGG19(include_top=False, pooling="max")

def get_img_features(image_path):
    img = image.load_img(img_path, target_size=(224, 224))
    x = image.img_to_array(img)
    x = np.expand_dims(x, axis=0)
    x = preprocess_input(x)   
    return img_model.predict(x)

img_df["img_features"] = img_df["img_path"].apply(get_img_features)

## Problem 2: Generating the word-feature space
This is another well-documented problem. We accomplish this using a basic word2vec model which generates a dense repreprentation of a one-hot encoded vocabulary. Our tags will be just a small subspace of the entire word-vector space.

In [ ]:
import gensim

word_model = gensim.models.Word2Vec.load_word2vec_format('./GoogleNews-vectors-negative300.bin', binary=True)

def get_word_features(word):
    return word_model[word]

tag_df["tag_features"] = tag_df["tags"].apply(get_word_features)

## Problem 3: Assigning tags to images
The key idea from Zhang, et al is that the list of relevant tags for an image lie along a single direction in this word-feature space. This single direction can then be modelled by something as simple as a linear regression model. The only issue is how do we find this principal direction? It works out that we can approximate the principal direction for each image by applying a ranking algorithm to a set of images and their known tags.

### Sub-Problem: Ranking Words
Zhang, et al use a basic linear ranking SVM to learn each of these principal directions

In [ ]:
import itertools
from sklearn import svm
def transform_pairwise(X, y):
    X_new = []
    y_new = []
    y = np.asarray(y)
    if y.ndim == 1:
        y = np.c_[y, np.ones(y.shape[0])]
    comb = itertools.combinations(range(X.shape[0]), 2)
    for k, (i, j) in enumerate(comb):
        if y[i, 0] == y[j, 0] or y[i, 1] != y[j, 1]:
            # skip if same target or different group
            continue
        X_new.append(X[i] - X[j])
        y_new.append(np.sign(y[i, 0] - y[j, 0]))
        # output balanced classes
        if y_new[-1] != (-1) ** k:
            y_new[-1] = - y_new[-1]
            X_new[-1] = - X_new[-1]
    return np.asarray(X_new), np.asarray(y_new).ravel()


class RankSVM(svm.LinearSVC):
    def fit(self, X, y):
        X_trans, y_trans = transform_pairwise(X, y)
        super(RankSVM, self).fit(X_trans, y_trans)
        return self

    def decision_function(self, X):
        return np.dot(X, self.coef_.ravel())

    def predict(self, X):
                if hasattr(self, 'coef_'):
            return np.argsort(np.dot(X, self.coef_.ravel()))
        else:
            raise ValueError("Must call fit() prior to predict()")

    def score(self, X, y):
        X_trans, y_trans = transform_pairwise(X, y)
        return np.mean(super(RankSVM, self).predict(X_trans) == y_trans)

### Learning the principal directions
Now that we have a way of approximating each principal direction we can solve the 3rd problem.

In [ ]:
from sklearn.linear_model import LinearRegression

img_df["p_dir"] = img_df["relevant_tags"].apply(lambda y: RankSVM.fit(tag_df["tag_features"].values,y).predict(tag_df["tag_features"].values,y))

w = LinearRegression().fit(img_df["img_features"],img_df["p_dir"])